# Глава 14. Итераторы и генераторы, ч. 1

## Итераторы: первое знакомство

Объект является **итерируемым** либо если он физически является последовательностью, либо если он является объектом, который воспроизводит по одному результату за раз в контексте инструментов выполнения итераций, таких как цикл `for`

### Протокол итераций: итераторы файлов

Один из самых простых способов понять, что такое итераторы,  – это посмотреть, как они работают со встроенными типами, такими как файлы.

Объекты открытых файлов имеют метод с именем `readline`, который читает по одной строке текста из файла за одно обращение – каждый раз, вызывая метод `readline`, мы перемещаемся к следующей строке. По достижении конца файла возвращается пустая строка, что может служить сигналом
для выхода из цикла.

Кроме того, файлы имеют также метод `__next__`, который производит практически тот же эффект, - всякий раз, когда его вызывают, он возвращает следующую строку. Единственное исключение - по достижении конца файла он возбуждает встроенное исключение `StopIteration` вместо пустой строки.

> **Протокол итераций** - объект реализует метод `__next__`, который возвращает следующие значения и возбуждает исключение `StopIteration` в конце серии результатов.

**Чтение файла построчно**

```
for line in open('data.txt'):
    print(line, end='')
```

### Выполнение итераций вручную: `iter` и `next`

> **Встроенная функция `next`** автоматически вызывает метод `__next__` объекта.

Для объекта `X` вызов `next(X)` аналогичен `X.__next__()`

С технической точки зрения итерационный протокол имеет еще одну сторону.

В самом начале цикл `for` получает итератор из итерируемого объекта, передавая его

> **встроенной функции `iter`**, которая возвращает объект, имеющий требуемый метод `__next__`.

Это станет более очевидным, если посмотреть, на то, как внутренние механизмы циклов `for` обрабатывают такие встроенные типы
последовательностей, как списки:

In [1]:
L = [1, 2, 3]
I = iter(L)     # получить объект-итератор
I.__next__()    # вызвать __next__, чтобы получить след. элемент

1

In [2]:
next(I)

2

In [3]:
I.__next__()

3

In [4]:
I.__next__()

StopIteration: 

>Списки и многие другие встроенные объекты не имеют собственных итераторов, потому что они поддерживают **возможность участия сразу в нескольких итерациях**. Чтобы начать итерации по таким объектам, необходимо предварительно вызвать функцию `iter`

Ручной способ итераций: имитация цикла `for`

In [5]:
L = [1, 2, 3]
I = iter(L)

while True:
    try:
        X = next(I)
    except StopIteration:
        break
    print(X**2, end=' ')

1 4 9 

Следует также отметить, что цикл `for` и другие средства выполнения итераций иногда могут действовать иначе, при работе с пользовательскими классами, многократно производя операцию индексирования объекта вместо использования протокола итераций.

### Другие итераторы встроенных типов

Кроме файлов и  фактических последовательностей, таких как списки, удобные итераторы также имеют и другие типы.

В последних версиях Python словари имеют итератор, который автоматически возвращает по одному ключу за раз в контексте итераций:

In [7]:
D = dict(zip('abc', [1, 2, 3]))
I = iter(D)
next(I)

'a'

In [8]:
next(I)

'b'

In [9]:
next(I)

'c'

In [10]:
next(I)

StopIteration: 

Поэтому не надо больше вызывать метод `keys`, чтобы выполнить обход ключей словаря:

In [11]:
D = dict(zip('abc', [1, 2, 3]))

for key in D:
    print(key, D[key])

a 1
b 2
c 3


## Генераторы списков: первое знакомство

### Основы генераторов списков

Генератор списка:

* начинается с  некоторого составленного нами выражения, которое использует введенную нами переменную цикла


* далее следует заголовок цикла `for`, в котором объявляется переменная цикла и итерируемый объект

In [12]:
L = [1, 2, 3, 4, 5]

# генератор списков
[x + 10 for x in L]

[11, 12, 13, 14, 15]

С технической точки зрения всегда можно обойтись без генераторов списков, потому что существует возможность создавать список результатов выражения вручную, с помощью цикла `for`. Фактически это и есть точное представление внутреннего механизма генератора списков.

In [13]:
res = []
for x in L:
    res.append(x + 10)
res

[11, 12, 13, 14, 15]

### Использование генераторов списков для работы с файлами

Всякий раз, когда мы заговариваем о выполнении операций над каждым элементом последовательности, мы попадаем в  сферу действий генераторов списков.

```
lines = [line.rstrip() for line in open('data.txt')]
```

Это выражение значительную часть работы выполняет неявно  – интерпретатор сканирует файл и автоматически собирает список результатов выполнения операции. Кроме того, это наиболее эффективный способ, потому что большая часть действий выполняется внутри интерпретатора Python, который работает наверняка быстрее, чем эквивалентная инструкция `for`.

Напомню еще раз, что при работе с большими файлами выигрыш в скорости от применения генераторов списков может оказаться весьма существенным.

### Расширенный синтаксис генераторов списков

В цикл `for`, вложенный в выражение, можно добавить оператор `if`,
чтобы отобрать результаты, для которых условное выражение дает истинное
значение.

In [14]:
[x**2 for x in range(10) if x % 2 == 0]

[0, 4, 16, 36, 64]

>Генераторы списков могут иметь вложенные циклы, оформленные в виде серии операторов `for`, каждый из которых может иметь ассоциированный с ним оператор `if`

In [15]:
[x + y for x in 'abc' for y in 'lmn']

['al', 'am', 'an', 'bl', 'bm', 'bn', 'cl', 'cm', 'cn']

In [16]:
res = []
for x in 'abc':
    for y in 'lmn':
        res.append(x + y)
res

['al', 'am', 'an', 'bl', 'bm', 'bn', 'cl', 'cm', 'cn']

In [17]:
L = [[1, 2, 3], [4, 5, 6]]
[i for sublst in L for i in sublst]

[1, 2, 3, 4, 5, 6]

In [18]:
res = []
for sublst in L:
    for i in sublst:
        res.append(i)
res

[1, 2, 3, 4, 5, 6]

### Другие контексты операций

Любой инструмент, использующий протокол итераций, автоматически сможет работать с любыми встроенными и пользовательскими классами, реализующими его поддержку.

Генераторы списков, оператор `in`, встроенная функция `map` и другие встроенные средства, такие как функции `sorted` и `zip`, также основаны на применении итерационного протокола.

* `sorted` - сортирует элементы итерируемого объекта

* `zip` - объединяет элементы итерируемых объектов

* `enumerate` - создает пары из элементов итерируемых объектов и их позиций

* `filter` - отбирает элементы, для которых указанная функция возвращает истинное значение

* `reduce` - выполняет указанную операцию, объединяя все элементы в итерируемом объекте

* `sum` - вычисляет сумму всех чисел в любом итерируемом объекте

* `any`, `all` - возвращают `True`, если любой (`any`) или все (`all`) элементы итерируемого объекта являются истинными значениями соответственно

* `min`, `max` - возвращают наибольший и наименьший элемент итерируемого объекта  соответственно.

* `list`, `tuple` - создают новые объекты из итерируемых объектов

* `join` - строковый метод, который вставляет подстроку между строками, содержащимися в итерируемом объекте

**`*arg`** - распаковывание значений коллекций в отдельные аргументы, в виде этой синтаксической конструкции можно передать любой итерируемый объект

In [19]:
def f(a, b, c, d):
    print(a, b, c, d, sep='&')
    
f(1, 2, 3, 4)

1&2&3&4


In [20]:
f([1, 2, 3, 4])

TypeError: f() missing 3 required positional arguments: 'b', 'c', and 'd'

In [21]:
f(*[1, 2, 3, 4])

1&2&3&4


>Можно даже выполнить обход строк в файле!
>
>**`f(*open('script1.py'))`**

Существует возможность использовать встроенную функцию `zip` для распаковывания упакованных кортежей, передавая ей результаты другого
вызова функции `zip`

In [22]:
X = (1, 2)
Y = (3, 4)

list(zip(X, Y))  # Упаковать кортежи: возвратит итерируемый объект

[(1, 3), (2, 4)]

In [23]:
A, B = zip(*zip(X, Y)) # Распаковать упакованные кортежи!
A, B

((1, 2), (3, 4))

## Новые итерируемые объекты в Python 3.0

Одно из фундаментальных отличий Python 3.0 от 2.X заключается в том, что **в версии 3.0 делается сильный акцент на использование итераторов**. 

Вдобавок к тому, что итераторы были ассоциированы со многими встроенными типами данных, такими как файлы и словари, в Python 3.0 методы `keys`, `values` и `items` словарей также возвращают итерируемые объекты, как и  встроенные функции `range`, `map`, `zip` и `filter`. Как было показано в предыдущем разделе, три последние функции не только возвращают итераторы, но и выполняют обработку данных в них. Все эти инструменты в Python 3.0 не создают списки с результатами, как в версии 2.6, а возвращают результаты по требованию.

### Итератор `range`

`range` - возвращает итератор, который генерирует список целых чисел в заданном диапазоне по требованию.

Объекты, возвращаемые функцией `range` в версии 3.0, поддерживают только возможность итераций, операцию обращения к  элементам по индексам и  функцию `len`. Они не поддерживают никакие другие операции над последовательностями.

In [24]:
R = range(10)
I = iter(R)
list(R)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [25]:
next(I)

0

In [26]:
next(I)

1

In [27]:
len(R)

10

In [28]:
R[-1]

9

In [29]:
R[:4]

range(0, 4)

In [30]:
next(I)  # обход продолжается, где закончился

2

### Итераторы `map`, `zip` и `filter`

В версии 3.0 все три функции не только принимают итерируемые объекты в виде аргументов, но и возвращают итерируемые объекты в виде результатов.

Однако, в отличие от функции `range`, **их результаты сами и являются этими итераторами**, –
>**после однократного получения отдельного результата этот результат исчезает**

Другими словами, вы не сможете на основе результатов организовать проход по этим результатам нескольких итераторов, который обеспечил бы возможность извлечения результатов в разных позициях.

In [31]:
M = map(abs, (-1, 0, 1))
M

In [32]:
next(M)

1

In [33]:
next(M)

0

In [34]:
next(M)

1

In [35]:
next(M)

StopIteration: 

In [36]:
# Теперь итератор пуст - возможен только один проход
for x in M:
    print(x)

In [37]:
# Чтобы выполнить второй проход, нужно снова создать итератор
M = map(abs, (-1, 0, 1))
for x in M:
    print(x)

1
0
1


**`zip`** действует точно так же

In [38]:
# zip также возвращает итератор, который позволяет выполнить только один проход

Z = zip((1, 2, 3), (10, 20, 30))
Z

In [39]:
list(Z)

[(1, 10), (2, 20), (3, 30)]

In [40]:
# результаты исчерпываются после первого прохода
for pair in Z: print(pair)

**`filter`** действует аналогичным образом

In [41]:
filter(bool, ['spam', '', 'ni'])

In [42]:
list(filter(bool, ['spam', '', 'ni']))

['spam', 'ni']

### Поддержка множественных и единственных итераторов

Многим интересно будет увидеть, в чем заключаются различия между объектом диапазона, возвращаемым функцией `range`, и объектами, возвращаемыми другими встроенными функциями, которые описываются в этом разделе, 

– результат функции `range` поддерживает функцию `len` и операцию доступа к элементам по индексу, но сам не является итератором (итератор можно получить вызовом функции `iter` и  использовать для выполнения итераций вручную);

кроме того, он поддерживает возможность применения к результату нескольких итераторов, которые отслеживают свое положение в  последовательности целых чисел независимо друг от друга:

In [43]:
R = range(3) # Объект диапазона позволяет получить множество итераторов
next(R)

TypeError: 'range' object is not an iterator

In [44]:
I1 = iter(R)
next(I1)

0

In [45]:
next(I1)

1

In [46]:
I2 = iter(R) # Два итератора для одного диапазона

In [47]:
next(I2)

0

In [48]:
next(I1) # I1 находится в другой позиции, не совпадающей с позицией I2

2

>Функции `zip`, `map` и `filter`, напротив, не поддерживают возможность получения различных активных итераторов для одного и того же результата:

In [49]:
Z = zip((1, 2, 3), (10, 11, 12))
I1 = iter(Z)
I2 = iter(Z)  # два итератора для одного и того же результата zip

In [50]:
next(I1)

(1, 10)

In [51]:
next(I1)

(2, 11)

In [52]:
next(I2)  # Позиции итераторов I2 и I1 совпадают!

(3, 12)

**Поддержка множественных итераторов обычно обеспечивается за счет создания новых объектов, которые могут передаваться функции `iter`**.

Если поддерживается единственный итератор, функция `iter` обычно возвращает тот же объект, который она получила

### Итераторы представления словарей

Методы `keys`, `values` и `items` словарей возвращают итерируемые объекты представлений, которые возвращают результаты по одному за раз вместо того, чтобы сразу создавать списки с  результатами.

Элементы представлений сохраняют физический порядок следования в словаре, и все изменения, выполняемые в словаре, отражаются на его
представлениях.

In [53]:
D = dict(a=1, b=2, c=3)
D

{'a': 1, 'b': 2, 'c': 3}

In [54]:
K = D.keys()
K

dict_keys(['a', 'b', 'c'])

In [55]:
next(K)  # представления не являются итераторами

TypeError: 'dict_keys' object is not an iterator

In [56]:
# из представлений можно получить итераторы и с их помощью выполнять
# итерации вручную, но они не поддерживают функцию len()
# и операцию доступа к элементам по индексу
I = iter(K)
next(I)

'a'

In [57]:
list(K)

['a', 'b', 'c']

In [58]:
D['d'] = 4
list(K)

['a', 'b', 'c', 'd']

Словари поддерживают собственные итераторы, которые возвращают последовательности ключей

In [59]:
for key in D:
    print(key, end=' ')

a b c d 